In [19]:
import pandas as pd

nlp = pd.read_csv('preprocess/nlp-process.csv')
nlp.head()


,Unnamed: 0,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,DeletionDate,Score,ViewCount,Body,...,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate,ContentLicense
0,0,8707624,1,8790222.0,NaN,2012-01-03 03:33:13,NaN,5,1252,came across term called explicit semantic anal...,...,NaN,2012-01-09 14:50:17,explicit semantic analysis,text similarity text-mining,1,0,1.0,NaN,NaN,CC BY-SA 3.0
1,1,8720410,1,8780172.0,NaN,2012-01-04 00:02:18,NaN,2,1095,wa kicking around idea breaking large group te...,...,NaN,2012-01-08 18:43:09,compressing text using recursive n gram,text compression storage n-gram,2,0,1.0,NaN,NaN,CC BY-SA 3.0
2,2,8721488,1,NaN,NaN,2012-01-04 02:55:43,NaN,-2,883,compare two document find degree similarity ne...,...,NaN,2013-01-23 21:18:21,degree similarity,text similarity text-mining,2,3,1.0,NaN,NaN,CC BY-SA 3.0
3,3,8748870,1,8749344.0,NaN,2012-01-05 19:56:04,NaN,2,1010,using python nltk written regex find word star...,...,2012-01-05 19:59:49,2012-01-05 21:16:54,change findall regex text nltk text findall regex,python regex nltk,1,0,3.0,NaN,NaN,CC BY-SA 3.0
4,4,8751071,1,NaN,NaN,2012-01-05 22:53:26,NaN,5,3085,latent dirichlet analysis research keep runnin...,...,2013-02-25 09:09:24,2018-05-16 04:13:38,convert one document per line blei lda c dtm f...,nlp dataform lda,4,3,3.0,NaN,NaN,CC BY-SA 3.0


In [20]:
nlp.drop(['OwnerUserId', 'OwnerDisplayName','LastEditorUserId', 'LastEditorDisplayName'],axis=1,inplace=True)


In [21]:
import time, math
#计算热门问题
def hot (Qviews, Qanswers, Qscore, Ascore, date_ask, date_active):
    date_compute = time.mktime(time.strptime('2020-10-22 00:00:00', '%Y-%m-%d %H:%M:%S'))
    Qage = round((date_compute - date_ask) / 3600)
    Qupdated = round((date_compute - date_active) / 3600)
    return (math.log10(Qviews) * 4 + Qanswers * Qscore / 5 + Ascore )/(pow((Qage + 1) - (Qage - Qupdated) / 2,1.5))

In [22]:
nlp_score = nlp[nlp['PostTypeId'] == 1]
nlp_answer = nlp[nlp['PostTypeId'] == 2]
answerScore = {}
for index, row in nlp_answer.iterrows():
    parentId = row['ParentId']
    score = answerScore.get(parentId, 0) + row['Score']
    answerScore[parentId] = score

In [23]:
nlp_score = nlp[nlp['PostTypeId'] == 1]
nlp_score['HotScore']='0'
# nlp_score['CreationDate'] = pd.to_datetime(nlp_score['CreationDate'])
# nlp_score['LastActivityDate'] = pd.to_datetime(nlp_score['LastActivityDate'])
nlp_score['ViewCount'].fillna(0, inplace=True)
nlp_score['AnswerCount'].fillna(0, inplace=True)
nlp_score['Score'].fillna(0, inplace=True)

import time
for index, row in nlp_score.iterrows():
    Qviews = row['ViewCount']
    Qanswers = row['AnswerCount']
    Qscore = row['Score']
    if (row['Id'] in answerScore):
        Ascore = answerScore[row['Id']]
    else:
        Ascore = 0
    date_ask = time.mktime(time.strptime(row['CreationDate'], '%Y-%m-%d %H:%M:%S'))
    date_active = time.mktime(time.strptime(row['LastActivityDate'], '%Y-%m-%d %H:%M:%S'))
    hotscore = hot(Qviews, Qanswers, Qscore, Ascore, date_ask, date_active)
    nlp_score.loc[index,'HotScore'] = hotscore
# 进行筛选，条件score>10,viewcount>1000,answercount>0
out = nlp_score[(nlp_score['Score']>10) & (nlp_score['ViewCount']>1000) &(nlp_score['AnswerCount']>0)]
# 进行降序排序
out.sort_values(by='HotScore', ascending=False, inplace=True)
out[0:100]

<ipython-input-23-8efc94b73a87>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out.sort_values(by='HotScore', ascending=False, inplace=True)


,Unnamed: 0,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,DeletionDate,Score,ViewCount,Body,...,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate,ContentLicense,HotScore
19773,19773,54334304,1,54409674.0,NaN,2019-01-23 19:24:21,NaN,31,44775,difference spacy load en core web sm spacy loa...,...,2020-09-10 19:45:29,spacy find model en core web sm window python ...,python-3.x spacy,11,2,8.0,NaN,NaN,CC BY-SA 4.0,0.000118164
23335,23335,58636587,1,NaN,NaN,2019-10-31 03:34:11,NaN,16,7533,know bert ha max length limit token acticle ha...,...,2020-09-02 12:11:33,use bert long text classification,nlp text-classification bert-language-model,5,0,3.0,NaN,NaN,CC BY-SA 4.0,9.25103e-05
1166,1166,16370583,1,NaN,NaN,2013-05-04 04:29:21,NaN,104,204759,difficult time using pip install almost anythi...,...,2020-08-12 01:51:46,pip issue installing almost library,python pip nltk easy-install,26,1,37.0,NaN,NaN,CC BY-SA 3.0,9.13132e-05
16650,16650,49964028,1,50342159.0,NaN,2018-04-22 08:33:02,NaN,42,55195,even though downloaded model load jalal goku e...,...,2020-06-19 14:55:25,spacy oserror find model en,nlp spacy,11,1,12.0,NaN,NaN,CC BY-SA 3.0,8.01011e-05
6200,6200,26570944,1,26575754.0,NaN,2014-10-26 07:52:51,NaN,98,113655,code import nltk data tokenizer nltk data load...,...,2020-08-01 02:50:32,resource u tokenizers punkt english pickle found,python unix nltk,17,1,14.0,NaN,NaN,CC BY-SA 3.0,7.86305e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13445,13445,45394949,1,NaN,NaN,2017-07-29 23:24:19,NaN,11,11188,want understand meant dimensionality word embe...,...,2019-12-30 13:06:24,dimensionality word embeddings,nlp terminology dimensionality-reduction word-...,6,1,6.0,NaN,NaN,CC BY-SA 3.0,1.24628e-05
9168,9168,38045290,1,39190391.0,NaN,2016-06-27 03:02:43,NaN,32,16521,result two different summary system sys sys re...,...,2020-06-24 09:31:49,text summarization evaluation bleu v rouge,nlp text-processing rouge bleu,3,0,10.0,NaN,NaN,CC BY-SA 4.0,1.24071e-05
15786,15786,48941648,1,NaN,NaN,2018-02-23 05:26:07,NaN,16,3952,given model e g gensim model word vec import w...,...,2019-08-14 03:55:22,remove word completely word vec model gensim,python dictionary word2vec gensim del,4,4,2.0,NaN,NaN,CC BY-SA 3.0,1.23962e-05
2612,2612,20290870,1,24119115.0,NaN,2013-11-29 17:33:07,NaN,41,65062,trying extract human name text doe anyone meth...,...,2020-01-02 22:45:42,improving extraction human name nltk,python nlp nltk,7,4,28.0,2019-09-16 14:07:54,NaN,CC BY-SA 3.0,1.23703e-05


In [25]:
out[0:100].to_csv('hotquestion/nlp-hotquestion.csv')

In [28]:
cv = pd.read_csv('preprocess/cv-process.csv')
cv.drop(['OwnerUserId', 'OwnerDisplayName','LastEditorUserId', 'LastEditorDisplayName'],axis=1,inplace=True)
cv_score = cv[cv['PostTypeId'] == 1]
cv_answer = cv[cv['PostTypeId'] == 2]
answerScore = {}
for index, row in cv_answer.iterrows():
    parentId = row['ParentId']
    score = answerScore.get(parentId, 0) + row['Score']
    answerScore[parentId] = score

cv_score['HotScore']='0'
cv_score['ViewCount'].fillna(0, inplace=True)
cv_score['AnswerCount'].fillna(0, inplace=True)
cv_score['Score'].fillna(0, inplace=True)

import time
for index, row in cv_score.iterrows():
    Qviews = row['ViewCount']
    Qanswers = row['AnswerCount']
    Qscore = row['Score']
    if (row['Id'] in answerScore):
        Ascore = answerScore[row['Id']]
    else:
        Ascore = 0
    date_ask = time.mktime(time.strptime(row['CreationDate'], '%Y-%m-%d %H:%M:%S'))
    date_active = time.mktime(time.strptime(row['LastActivityDate'], '%Y-%m-%d %H:%M:%S'))
    hotscore = hot(Qviews, Qanswers, Qscore, Ascore, date_ask, date_active)
    cv_score.loc[index,'HotScore'] = hotscore
# 进行筛选，条件score>10,viewcount>1000,answercount>0
cvout = cv_score[(cv_score['Score']>10) & (cv_score['ViewCount']>1000) &(cv_score['AnswerCount']>0)]
# 进行降序排序
cvout.sort_values(by='HotScore', ascending=False, inplace=True)
cvout[0:100].to_csv('hotquestion/cv-hotquestion.csv')

D:\anaconda3\envs\lda\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
<ipython-input-28-468bc15a41fa>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cvout.sort_values(by='HotScore', ascending=False, inplace=True)


In [29]:
speech = pd.read_csv('preprocess/speech-process.csv')
speech.drop(['OwnerUserId', 'OwnerDisplayName','LastEditorUserId', 'LastEditorDisplayName'],axis=1,inplace=True)
speech_score = speech[speech['PostTypeId'] == 1]
speech_answer = speech[speech['PostTypeId'] == 2]
answerScore = {}
for index, row in speech_answer.iterrows():
    parentId = row['ParentId']
    score = answerScore.get(parentId, 0) + row['Score']
    answerScore[parentId] = score

speech_score['HotScore']='0'
speech_score['ViewCount'].fillna(0, inplace=True)
speech_score['AnswerCount'].fillna(0, inplace=True)
speech_score['Score'].fillna(0, inplace=True)

import time
for index, row in speech_score.iterrows():
    Qviews = row['ViewCount']
    Qanswers = row['AnswerCount']
    Qscore = row['Score']
    if (row['Id'] in answerScore):
        Ascore = answerScore[row['Id']]
    else:
        Ascore = 0
    date_ask = time.mktime(time.strptime(row['CreationDate'], '%Y-%m-%d %H:%M:%S'))
    date_active = time.mktime(time.strptime(row['LastActivityDate'], '%Y-%m-%d %H:%M:%S'))
    hotscore = hot(Qviews, Qanswers, Qscore, Ascore, date_ask, date_active)
    speech_score.loc[index,'HotScore'] = hotscore
# 进行筛选，条件score>10,viewcount>1000,answercount>0
speechout = speech_score[(speech_score['Score']>10) & (speech_score['ViewCount']>1000) &(speech_score['AnswerCount']>0)]
# 进行降序排序
speechout.sort_values(by='HotScore', ascending=False, inplace=True)
speechout[0:100].to_csv('hotquestion/speech-hotquestion.csv')


<ipython-input-29-3c41836f8b09>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  speechout.sort_values(by='HotScore', ascending=False, inplace=True)
